In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import trend_finder_lr as tflr
import trend_finder_ovr as tfovr
import ml_models as ml
import trend_finder_ovr as ovr
import yfinance as yf
import pickle
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, svm
from imblearn.over_sampling import RandomOverSampler

# Run a test looking for stock price increases and decreases of percent, 30 days out
- Record 60 trading days before the trend
- use those 60 trading days as features

In [2]:
tesla = tflr.Historydf('TSLA', '1y', 30, 5, 60)

In [3]:
tesla.df

,Open,Close,Volume
0,302.359985,296.070007,53230000
1,297.429993,288.089996,57163900
2,282.829987,284.820007,41864700
3,287.869995,277.700012,50541800
4,280.619995,275.609985,52107300
...,...,...,...
246,214.119995,215.490005,135813700
247,221.550003,231.279999,135702700
248,240.250000,233.190002,130597900
249,229.339996,236.860001,101077600


# Find trends, enter [Buy/Sell] classifier, and create new dataframe

In [4]:
tesla.find_trends()

start index = 59, end index = 221


In [5]:
tesla.pretrends_df

,Open,Close,Volume,Buy/Sell
0,238.660004,230.039993,99085249.0,NaN
1,302.359985,296.070007,53230000.0,NaN
2,297.429993,288.089996,57163900.0,NaN
3,282.829987,284.820007,41864700.0,NaN
4,287.869995,277.700012,50541800.0,NaN
...,...,...,...,...
7675,278.089996,276.540009,120332100.0,NaN
7676,278.429993,274.429993,113602000.0,NaN
7677,276.470001,269.609985,119425400.0,NaN
7678,268.649994,269.790009,91972400.0,NaN


# Structure the data in preperation for training model and normalize the values

In [6]:
tesla.structure_data()

In [7]:
tesla.structured_df

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,Buy/Sell
0,0.464481,0.948965,0.911469,0.800426,0.838759,0.783617,0.722467,0.787040,0.723228,0.726422,...,0.624819,0.573016,0.957071,0.937908,1.000000,0.798550,0.554413,0.544144,0.271944,0
1,0.948965,0.911469,0.800426,0.838759,0.783617,0.722467,0.787040,0.723228,0.726422,0.788789,...,0.573016,0.957071,0.937908,1.000000,0.798550,0.554413,0.544144,0.271944,0.247377,0
2,0.911469,0.800426,0.838759,0.783617,0.722467,0.787040,0.723228,0.726422,0.788789,0.867661,...,0.957071,0.937908,1.000000,0.798550,0.554413,0.544144,0.271944,0.247377,0.376319,0
3,0.814164,0.849858,0.798513,0.741572,0.801700,0.742280,0.745255,0.803328,0.876771,0.940864,...,0.937908,1.000000,0.798550,0.554413,0.544144,0.271944,0.247377,0.376319,0.561635,0
4,0.849858,0.798513,0.741572,0.801700,0.742280,0.745255,0.803328,0.876771,0.940864,0.885482,...,1.000000,0.782224,0.518301,0.507200,0.212940,0.186382,0.325774,0.526109,0.354377,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0.262246,0.293438,0.233518,0.241143,0.259396,0.265789,0.203789,0.104205,0.093654,0.092498,...,0.713324,0.741057,0.618757,0.576444,0.344522,0.189711,0.250105,0.346537,0.255366,0
124,0.293438,0.233518,0.241143,0.259396,0.265789,0.203789,0.104205,0.093654,0.092498,0.055299,...,0.741057,0.618757,0.576444,0.344522,0.189711,0.250105,0.346537,0.255366,0.200575,0
125,0.233518,0.241143,0.259396,0.265789,0.203789,0.104205,0.093654,0.092498,0.055299,0.058919,...,0.618757,0.576444,0.344522,0.189711,0.250105,0.346537,0.255366,0.200575,0.247984,0
126,0.241143,0.259396,0.265789,0.203789,0.104205,0.093654,0.092498,0.055299,0.058919,0.000000,...,0.576444,0.344522,0.189711,0.250105,0.346537,0.255366,0.200575,0.247984,0.024484,0


# Run logistic regression

In [8]:
tesla_lr = ml.logistic_regression('Tesla')

In [9]:
tesla_lr.train_model(tesla.structured_df)

The testing accuracy is: 1.00
Coefficients:
 [[-1.04135315e-02 -7.67362161e-02 -4.82426616e-02  9.34696022e-02
   1.26042046e-01  1.34794808e-01  1.56907909e-01  2.17042472e-01
   2.06437776e-01  1.34681718e-01  6.21439133e-02  3.42366796e-02
   4.98148874e-02  1.03692133e-01  5.69153148e-02 -2.50000444e-02
  -7.37757352e-02 -6.19102144e-02 -1.06802178e-01  8.30188159e-02
   1.00110249e-01  2.03546374e-01  2.51178630e-01  1.89663274e-01
   1.64090996e-01  2.15114317e-01  2.74911915e-01  2.86891665e-01
   2.05485014e-01  1.49560821e-01  1.28824673e-01  7.88943709e-02
   1.17659673e-01  6.89299101e-02  4.18067784e-03 -1.51840595e-01
  -1.57438152e-01 -8.29736029e-02 -1.75053886e-01 -1.43797347e-01
  -6.77342443e-03 -1.10662253e-01 -1.54936423e-01 -1.90307743e-01
  -2.50971637e-01 -2.51173971e-01 -2.29971863e-01 -2.21684933e-01
  -1.77560040e-01 -1.18372004e-01 -9.31074510e-02 -1.40086660e-01
  -5.63705892e-02  5.77532587e-03 -6.28733990e-02 -6.82124118e-02
  -4.79555746e-02 -6.85027179e-

# Adding a third classification for hold
- hold is represented by 2
- will include any points without increases or decreases of desired percentage
- model being used SVM

In [10]:
tesla = ovr.Historydf('TSLA', '5y', 10, 5, 60)

In [11]:
tesla.find_trends()

start index = 59, end index = 1247


In [12]:
tesla.pretrends_df

,Open,Close,Volume,Buy/Sell
0,238.660004,230.039993,99085249.0,NaN
1,21.200001,21.284666,196189500.0,NaN
2,21.227333,20.790667,114736500.0,NaN
3,20.684668,20.334000,111711000.0,NaN
4,20.150667,20.209999,108250500.0,NaN
...,...,...,...,...
71275,252.039993,259.320007,97569100.0,NaN
71276,260.970001,253.860001,99242600.0,NaN
71277,251.449997,251.449997,111097900.0,NaN
71278,247.449997,249.699997,96642200.0,NaN


In [13]:
tesla.structure_data()

In [14]:
tesla.structured_df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,Buy/Sell
0,1.000000,0.020256,0.020379,0.017934,0.015529,0.015451,0.013931,0.010359,0.010284,0.002865,...,0.078209,0.099177,0.089447,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,2
1,0.637550,0.641425,0.564473,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,...,0.099177,0.089447,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,0.179210,2
2,0.641425,0.564473,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,...,0.089447,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,0.179210,0.120783,1
3,0.564473,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,...,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,0.179210,0.120783,0.007032,1
4,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,...,0.093011,0.042313,0.028448,0.014368,0.102000,0.186990,0.129116,0.016443,0.000000,1
5,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,...,0.042313,0.028448,0.014368,0.102000,0.186990,0.129116,0.016443,0.000000,0.128688,1
6,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,0.373227,...,0.028448,0.014368,0.102000,0.186990,0.129116,0.016443,0.000000,0.128688,0.073206,1
7,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,0.373227,0.436094,...,0.016872,0.104281,0.189055,0.131329,0.018942,0.002540,0.130902,0.075560,0.000000,1
8,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,0.373227,0.436094,0.283135,...,0.134957,0.216828,0.161078,0.052540,0.036701,0.160666,0.107220,0.034247,0.000000,2
9,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,0.373227,0.436094,0.283135,0.501040,...,0.216828,0.161078,0.052540,0.036701,0.160666,0.107220,0.034247,0.000000,0.083366,0


In [17]:
tesla_lr = ml.svm_ovr('Tesla')

# Note: accuracy is higher in Spyder

In [18]:
acc = 0
mod = svm.SVC()

for i in range(0, 200):
    tesla_lr.train_model(tesla.structured_df)
    print(acc)
    if tesla_lr.accuracy > acc:
        acc = tesla_lr.accuracy
        mod = tesla_lr.model
tesla_lr.model = mod
tesla_lr.accuracy = acc
tesla_lr.save_current_model()

The testing accuracy is: 0.57
0
The testing accuracy is: 0.59
0.5658263305322129
The testing accuracy is: 0.63
0.5938375350140056
The testing accuracy is: 0.59
0.6274509803921569
The testing accuracy is: 0.59
0.6274509803921569
The testing accuracy is: 0.60
0.6274509803921569
The testing accuracy is: 0.61
0.6274509803921569
The testing accuracy is: 0.61
0.6274509803921569
The testing accuracy is: 0.61
0.6274509803921569
The testing accuracy is: 0.57
0.6274509803921569
The testing accuracy is: 0.65
0.6274509803921569
The testing accuracy is: 0.57
0.6470588235294118
The testing accuracy is: 0.58
0.6470588235294118
The testing accuracy is: 0.60
0.6470588235294118
The testing accuracy is: 0.56
0.6470588235294118
The testing accuracy is: 0.60
0.6470588235294118
The testing accuracy is: 0.58
0.6470588235294118
The testing accuracy is: 0.56
0.6470588235294118
The testing accuracy is: 0.63
0.6470588235294118
The testing accuracy is: 0.61
0.6470588235294118
The testing accuracy is: 0.59
0.64705

The testing accuracy is: 0.59
0.6582633053221288
The testing accuracy is: 0.62
0.6582633053221288
The testing accuracy is: 0.57
0.6582633053221288
The testing accuracy is: 0.61
0.6582633053221288
The testing accuracy is: 0.57
0.6582633053221288
The testing accuracy is: 0.60
0.6582633053221288
The testing accuracy is: 0.61
0.6582633053221288
The testing accuracy is: 0.57
0.6582633053221288
The testing accuracy is: 0.61
0.6582633053221288
The testing accuracy is: 0.60
0.6582633053221288
The testing accuracy is: 0.60
0.6582633053221288
The testing accuracy is: 0.64
0.6582633053221288
The testing accuracy is: 0.62
0.6582633053221288
The testing accuracy is: 0.57
0.6582633053221288
The testing accuracy is: 0.65
0.6582633053221288
The testing accuracy is: 0.61
0.6582633053221288
The testing accuracy is: 0.60
0.6582633053221288
The testing accuracy is: 0.60
0.6582633053221288
The testing accuracy is: 0.57
0.6582633053221288
The testing accuracy is: 0.62
0.6582633053221288
The testing accuracy